In [1]:
import numpy as np
import cPickle as pickle
from itertools import groupby
import random as random
import matplotlib.pyplot as plt
%matplotlib inline

from knn_functions import *

In [2]:
# load data
training_data = pickle.load(open('visit_182','rb'))
test_data = pickle.load(open('test_182','rb'))
# ground truth, the last element of list 
label = map(lambda x:x[-1],test_data)

print len(training_data),len(test_data)

5076 1270


In [3]:
# cross validation
K_FOLD = 4

SAMPLE_SIZE = len(training_data)
FOLD_SIZE = SAMPLE_SIZE / K_FOLD + 1
random.seed(10000)
random_index = random.sample(range(SAMPLE_SIZE),SAMPLE_SIZE)
index_range = [(i*FOLD_SIZE, min((i+1)*FOLD_SIZE,SAMPLE_SIZE)) for i in range(K_FOLD)]
print index_range

[(0, 1270), (1270, 2540), (2540, 3810), (3810, 5076)]


In [4]:
def CV(data,index_range,knn_set,top_k_set):
    k_fold = len(index_range)
    max_knn = max(knn_set)
    accuracy_CV = []
    for k in range(k_fold):
        print 'process %d th fold CV' % k
        index = index_range[k]
        train = data[:index[0]] + data[index[1]:]
        valid = data[index[0]:index[1]]
        sample_size = len(train)
        
        valid_label = map(lambda x:x[-1],valid)
        
        neighbor_index = getNeighbors(valid,train,max_knn)
        
        for knn in knn_set:
            print '%d nearest neighbors' % knn
            prediction = getPrediction(train,neighbor_index[:,:knn])
            for top_k in top_k_set:
                top_k_prediction = getTopK(prediction,top_k)
                acc = getAccuracy(top_k_prediction,valid_label)
                accuracy_CV.append([k,knn,top_k,np.mean(acc),np.std(acc)])
    return accuracy_CV

In [5]:
# parameters 
KNN = [20,30,40,50] # K nearest neighbors
TOP_K = [10,20,30] # prediction on TOP_K codes

accuracy_CV = CV(training_data,index_range,KNN,TOP_K)

process 0 th fold CV
20 nearest neighbors
30 nearest neighbors
40 nearest neighbors
50 nearest neighbors
process 1 th fold CV
20 nearest neighbors
30 nearest neighbors
40 nearest neighbors
50 nearest neighbors
process 2 th fold CV
20 nearest neighbors
30 nearest neighbors
40 nearest neighbors
50 nearest neighbors
process 3 th fold CV
20 nearest neighbors
30 nearest neighbors
40 nearest neighbors
50 nearest neighbors


In [6]:
accuracy_CV = np.asarray(accuracy_CV)
accuracy_result = {}
for acc in accuracy_CV:
    knn,top_k = acc[1:3]
    if (knn,top_k) in accuracy_result.keys():
        accuracy_result[(knn,top_k)] = np.add(accuracy_result[(knn,top_k)],acc[3:])
    else:
        accuracy_result[(knn,top_k)] = acc[3:]
accuracy_result = np.asarray(accuracy_result.items())
accuracy_result[:,1] = accuracy_result[:,1]/K_FOLD

In [7]:
print sorted(accuracy_result,key=lambda x:(x[0,0],x[0,1]))

[array([[ 20.        ,  10.        ],
       [  0.38111156,   0.17762095]]), array([[ 20.        ,  20.        ],
       [  0.53154085,   0.18500657]]), array([[ 20.        ,  30.        ],
       [  0.62636567,   0.17953687]]), array([[ 30.        ,  10.        ],
       [  0.38243024,   0.1775758 ]]), array([[ 30.        ,  20.        ],
       [  0.53771719,   0.18354634]]), array([[ 30.        ,  30.        ],
       [  0.63478873,   0.17744267]]), array([[ 40.        ,  10.        ],
       [  0.38326788,   0.1769825 ]]), array([[ 40.        ,  20.        ],
       [  0.5405609 ,   0.18474115]]), array([[ 40.        ,  30.        ],
       [  0.63797577,   0.17819837]]), array([[ 50.        ,  10.        ],
       [  0.38176919,   0.17655954]]), array([[ 50.        ,  20.        ],
       [  0.54171335,   0.18554043]]), array([[ 50.        ,  30.        ],
       [  0.63768278,   0.17900831]])]
